In [ ]:
# Setup
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import load_model
from skimage.color import lab2rgb
from PIL import Image
import io
from google.colab import files
import urllib.request
import os

# Downloading and Loading Models
model_url = "https://github.com/shaswat373/Image-Colorizer/raw/main/colorizer_model.h5"
model_path = "colorizer_model.h5"

if not os.path.exists(model_path):
    print("Downloading model from GitHub...")
    urllib.request.urlretrieve(model_url, model_path)
    print("Model downloaded.")

model = load_model(model_path, compile=False)
print("Model loaded successfully.")

# Image Upload
print("Please upload a grayscale image (32x32)...")
uploaded = files.upload()

for filename in uploaded:
    # Load image and convert to grayscale
    image = Image.open(io.BytesIO(uploaded[filename])).convert('L')
    image = image.resize((32, 32))
    l_channel = np.array(image).astype('float32') / 255.0  # normalize to [0,1]
    l_channel = l_channel.reshape(1, 32, 32, 1)  # reshape for model

    ab_pred = model.predict(l_channel)[0] * 128.0  # unnormalize ab

    lab_image = np.zeros((32, 32, 3))
    lab_image[:, :, 0] = l_channel[0, :, :, 0] * 100.0  # unnormalize L
    lab_image[:, :, 1:] = ab_pred
    rgb_result = lab2rgb(lab_image)

    # Display
    plt.figure(figsize=(8, 4))

    # Grayscale input
    plt.subplot(1, 2, 1)
    plt.imshow(image, cmap='gray')
    plt.title("Grayscale Input")
    plt.axis('off')

    # Colorized output
    plt.subplot(1, 2, 2)
    plt.imshow(rgb_result)
    plt.title("Colorized Output")
    plt.axis('off')

    plt.tight_layout()
    plt.show()
